In [1]:
from src.guitar.Guitar import Guitar
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.guitar.GuitarString import createGuitarStrings
from src.midi.midiToNote import midiToGuitarNotes,processedNotes
from src.utils.utils import convertNotesToChord, convertChordTofingerPositions
from src.HandPoseRecorder import HandPoseRecorder,HandPoseRecordPool
import copy
import time

## Import Midi file 导入Midi文件
Import midi file and convert it to a list of dict which including notes and beat. 导入midi文件，并将其转换为一个列表，这个列表里的元素是包含音符和节拍信息的字典。

In [2]:
midiFilePath = "asset/Aguado_12valses_Op1_No2.mid"
filename = midiFilePath.split("/")[-1].split(".")[0]

GuitarNotes = midiToGuitarNotes(midiFilePath)

GuitarNotes

[{'notes': [48, 52, 55], 'beat': 0.375},
 {'notes': [57], 'beat': 0.5},
 {'notes': [55], 'beat': 1.0},
 {'notes': [64], 'beat': 1.5},
 {'notes': [48, 60], 'beat': 2.0},
 {'notes': [55], 'beat': 2.5},
 {'notes': [52, 55], 'beat': 3.0},
 {'notes': [47, 53, 55], 'beat': 3.375},
 {'notes': [57], 'beat': 3.5},
 {'notes': [55], 'beat': 4.0},
 {'notes': [65], 'beat': 4.5},
 {'notes': [47, 62], 'beat': 5.0},
 {'notes': [55], 'beat': 5.5},
 {'notes': [53, 55], 'beat': 6.0},
 {'notes': [48, 55, 64], 'beat': 6.375},
 {'notes': [65], 'beat': 6.5},
 {'notes': [64], 'beat': 7.0},
 {'notes': [47, 56, 62], 'beat': 7.5},
 {'notes': [45, 57, 60], 'beat': 7.875},
 {'notes': [62], 'beat': 8.0},
 {'notes': [60], 'beat': 8.5},
 {'notes': [40, 55, 58], 'beat': 9.0},
 {'notes': [41, 53, 57], 'beat': 9.5},
 {'notes': [62], 'beat': 10.0},
 {'notes': [42, 51, 60], 'beat': 10.5},
 {'notes': [43, 50, 60], 'beat': 10.75},
 {'notes': [57], 'beat': 11.25},
 {'notes': [55], 'beat': 11.5},
 {'notes': [54], 'beat': 11.7

## init guitar and handPoseRecordPool 初始化吉他和handPoseRecordPool
init guitar with a list of string. here we use a standard 6-string guitar. 用一个字符串列表来初始化吉他。这里我们使用标准的6弦吉他。
init the start hand pose. here we put a hand at barrel 0 and put fingers on string 3, which is G string. 初始化起始手型。在这里，我们将设定初始手型位置为第0品，并将手指放在第3弦上，也就是G弦上。

handPoseRecordPool can init with param `size`, which is the size of the pool. Bigger size will cost more memory and more time but can calculate more possibilities. handPoseRecordPool可以通过参数`size`初始化，即池的大小。越大就会消耗更多的内存和时间，但可以计算更多可能性。

In [3]:
# 设定各弦音高
guitar_string_list = createGuitarStrings(["e", "b", "G", "D", "A", "E1"])
# 初始化吉它
guitar = Guitar(guitar_string_list)
# 设定各手指状态
leftFingers = [
    LeftFinger(1, guitar_string_list[1], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[3], 3),
    LeftFinger(4, guitar_string_list[4], 4)
]
# 初始化左手
initLeftHand = LeftHand(leftFingers)
# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0,0)
# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(20)
handPoseRecordPool.append(handPoseRecord)

## Calculate the possibility of hand pose 计算手型的可能性
- For each notes, we calculate all possibile chords, which includes position infomations of notes on guitar. 对于每个音符，我们计算所有可能的和弦，包括音符在吉他上的位置信息。
- For each chords, we calculate all possibile hand poses, which includes the position of fingers on guitar.
- For each hand poses, we generate a LeftHand object if it is possible to play. 对于每个和弦，我们计算所有可能的手型，包括手指在吉他上的位置。
- For each LeftHand object, we calculate entropy, the cost of switch hand pose. Then we append it to the handPoseRecord. 对于每个手型，如果可以演奏，我们就生成一个LeftHand对象。
- For each handPoseRecord, we put it in the handPoseRecordPool if its entropy is lower than the record in the pool. 对于每个LeftHand对象，我们计算熵，即切换手型的成本。然后将其添加到handPoseRecord中。
- After all notes are calculated, we get the best handPoseRecord from the pool. Then print the result. 计算完所有音符后，我们从池中获取最佳handPoseRecord，然后打印结果


In [4]:
for i in range(len(GuitarNotes)):
    start_time = time.time()
    guitarNote = GuitarNotes[i]
    notes = guitarNote["notes"]
    beat = guitarNote["beat"]
    notes = processedNotes(notes)
    
    # calculate all possible chords, including the position information of notes on the guitar. 计算所有可能的和弦,包含音符在吉它上的位置信息。
    chords = convertNotesToChord(notes, guitar)
    
    # init current record list. 记录池先更新初始化当前记录列表。
    handPoseRecordPool.readyForRecord()
        
    # calculate all possible fingerings, including the position information of fingers on the guitar. 计算所有可能的按法，包含手指在吉它上的位置信息。
    for chord in chords:
        fingerPositionsList = convertChordTofingerPositions(chord)
        for fingerPositions in fingerPositionsList:
            # Iterate through all recorders in the record pool. 遍历记录池中的所有记录器。
            for handPoseRecord in handPoseRecordPool.preHandPoseRecordPool:
                oldhand = handPoseRecord.currentHandPose()
                # Iterate through the list of fingerings, generate a new LeftHand object based on the fingering. 遍历按法列表，根据按法生成新的LeftHand对象。
                hand, entropy = oldhand.generateNextHands(
                    guitar, fingerPositions)
                if hand is None:
                    continue
                newHandPoseRecord = copy.deepcopy(handPoseRecord)
                newHandPoseRecord.addHandPose(hand, entropy, beat)
                handPoseRecordPool.append(newHandPoseRecord)
    
    end_time = time.time()
    print(f"当前beat是{beat}，当前有{len(handPoseRecordPool.curHandPoseRecordPool)}个rerocder，耗时{end_time-start_time:.2f}秒。")


# after all iterations, read the best solution in the record pool. 全部遍历完以后，读取记录池中的最优解。
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.output(True)
bestHandPoseRecord.save(f"output/{filename}.json")

当前beat是0.375，当前有3个rerocder，耗时0.00秒。
当前beat是0.5，当前有4个rerocder，耗时0.00秒。
当前beat是1.0，当前有4个rerocder，耗时0.01秒。
当前beat是1.5，当前有3个rerocder，耗时0.01秒。
当前beat是2.0，当前有11个rerocder，耗时0.01秒。
当前beat是2.5，当前有5个rerocder，耗时0.02秒。
当前beat是3.0，当前有8个rerocder，耗时0.02秒。
当前beat是3.375，当前有17个rerocder，耗时0.03秒。
当前beat是3.5，当前有4个rerocder，耗时0.05秒。
当前beat是4.0，当前有2个rerocder，耗时0.01秒。
当前beat是4.5，当前有2个rerocder，耗时0.01秒。
当前beat是5.0，当前有8个rerocder，耗时0.01秒。
当前beat是5.5，当前有4个rerocder，耗时0.03秒。
当前beat是6.0，当前有11个rerocder，耗时0.03秒。
当前beat是6.375，当前有20个rerocder，耗时0.22秒。
当前beat是6.5，当前有3个rerocder，耗时0.14秒。
当前beat是7.0，当前有3个rerocder，耗时0.02秒。
当前beat是7.5，当前有5个rerocder，耗时0.03秒。
当前beat是7.875，当前有6个rerocder，耗时0.04秒。
当前beat是8.0，当前有17个rerocder，耗时0.05秒。
当前beat是8.5，当前有3个rerocder，耗时0.16秒。
当前beat是9.0，当前有20个rerocder，耗时0.03秒。
当前beat是9.5，当前有2个rerocder，耗时0.03秒。
当前beat是10.0，当前有6个rerocder，耗时0.02秒。
当前beat是10.5，当前有5个rerocder，耗时0.02秒。
当前beat是10.75，当前有10个rerocder，耗时0.07秒。
当前beat是11.25，当前有5个rerocder，耗时0.09秒。
当前beat是11.5，当前有1个rerocder，耗时0.04秒。
当前beat是11.75，当前有4个rerocder


## Initialize Animation Values 初始化动画所需数值

In Blender, the position and orientation of a character's arm and hand are determined by several key values:
在blender中，人物的手臂和手掌状态，是由以下几个关键值决定的：
- The position of the palm's IK controller, which determines the position of the hand. 手掌的ik控制器的位置，它决定了手掌的位置。
- The pivot point ik_pivot of the palm IK controller, which determines the state of the arm and forearm. 手掌ik控制器的轴点ik_pivot，它决定了两节手臂的状态。
- The rotation of the palm bone, which determines the orientation of the hand. Additionally, because there are constraints on the deformation bones of the arm, it also affects the rotation of the arm and forearm. 手掌骨的rotation，它决定了手掌的朝向。同时因为手臂变形骨骼上有约束器，它还会影响两节手臂的旋转。

By recording several key values in four extreme states, we can interpolate and calculate the arm and hand state for any position. These four extreme states are:
只要记录四个极端状态下的几个关键值，我们就可以插值计算出任何状态下的手臂和手掌状态。这四个极端状态是：
- Hand in first position, with all fingers pressing the first string. 手在第一把位，并且手指全按第一弦的状态.
- Hand in first position, with all fingers pressing the sixth string. 手在第一把位，并且手指全按第六弦的状态.
- Hand in twelfth position, with the index finger pressing the first string at the twelfth fret. 手在第十二把位，并且食指按1弦12品的状态.
- Hand in twelfth position, with the index finger pressing the sixth string at the twelfth fret. 手在第十二把位，并且食指按6弦12品的状态

The position of the thumb (thumb_ik_position) is determined by the corresponding neck midline positions when the hand is in the first and twelfth positions. 大拇指的位置(thumb_ik_position)，是由手在第一把位和第十二把位时，大拇指所对应的琴颈中线位置所决定。

The position of each finger (finger_ik_position) can be calculated by interpolating the positions from four extreme states:
每个手指的位置(finger_ik_position)，是由四个极端状态下手指位置进行插件计算可以得到的。这四个极端状态是：
- Open string position on the first string. 1弦空品的位置
- Open string position on the sixth string. 6弦空品的位置
- Twelfth fret position on the first string. 1弦12品的位置
- Twelfth fret position on the sixth string. 6弦12品的位置

Finally, we need to know the BPM of the music and the FPS of the animation in Blender software to calculate exactly which frame to insert which hand shape. 最后，我们还需要知道乐曲的BPM，还有blender软件中动画的FPS，就能计算出到底在哪一帧应该插入哪个手型。

In [5]:
from numpy import array

# 记录blender文件中四个关键状态下，左手IK的位置,旋转值,左手Ik-Pivot的位置，finger_position的位置
# 状态一：左手在0把位，左手食指按1弦，将人物调整成合适的状态，记录下左手的位置,旋转值,左手Ik-Pivot的位置，finger_position的位置
# 注意这里的finger_position_p0并不是食指的位置，而是1弦与琴码的交点位置
hand_ik_position_0 = array([-0.09394, -0.224268, 0.285955])
hand_local_rotation_0 = array([94.9, 13.9, -101])
hand_ik_pivot_0 = array([0.11056, -0.016219, -0.052158])
finger_position_0 = array([0.600535, -0.202619, 1.7272])

state_0 = {
    "hand_ik_position": hand_ik_position_0,
    "hand_local_rotation": hand_local_rotation_0,
    "hand_ik_pivot": hand_ik_pivot_0,
    "finger_position": finger_position_0
}

# 状态二：左手在0把，左手手指按6弦
hand_ik_position_1 = array([-0.09394, -0.224268, 0.285955])
hand_local_rotation_1 = array([94.9, 13.9, -101])
hand_ik_pivot_1 = array([0.11056, -0.016219, -0.052158])
finger_position_1 = array([0.570612, -0.197357, 1.76472])

state_1 = {
    "hand_ik_position": hand_ik_position_1,
    "hand_local_rotation": hand_local_rotation_1,
    "hand_ik_pivot": hand_ik_pivot_1,
    "finger_position": finger_position_1
}

# 状态三：左手在12把，左手手指按1弦12品
hand_ik_position_2 = array([0.243842, -0.312618, -0.05911])
hand_local_rotation_2 = array([96.6, 4.54, -106])
hand_ik_pivot_2 = array([0.11056, -0.016219, -0.052158])
finger_position_2 = array([0.322629, -0.284379, 1.50895])

state_2 = {
    "hand_ik_position": hand_ik_position_2,
    "hand_local_rotation": hand_local_rotation_2,
    "hand_ik_pivot": hand_ik_pivot_2,
    "finger_position": finger_position_2
}

# 状态三：左手在12把，左手手指按6弦12品
hand_ik_position_3 = array([0.243842, -0.312618, -0.05911])
hand_local_rotation_3 = array([96.6, 4.54, -106])
hand_ik_pivot_3 = array([0.11056, -0.016219, -0.052158])
finger_position_3 = array([0.288106, -0.290331, -0.290331])

state_3 = {
    "hand_ik_position": hand_ik_position_3,
    "hand_local_rotation": hand_local_rotation_3,
    "hand_ik_pivot": hand_ik_pivot_3,
    "finger_position": finger_position_3
}

# 记录琴颈中线的p0和p12的位置，好放置大拇指
neck_position_p0 = array([0.11056, -0.016219, -0.052158])
neck_position_p12 = array([0.11056, -0.016219, -0.052158])
neck_positions = {
    "start": neck_position_p0,
    "end": neck_position_p12
}

# beats per minute. 乐曲的演奏速度
BPM = 100
# frames per second. blender文件中的动画帧率
FPS = 24

recorder = f"output/{filename}.json"
animation = f"output/{filename}_animation.json"

In [6]:
from src.animate.animate import hand2Animation
hand2Animation(recorder, animation, BPM, FPS, state_0, state_1, state_2, state_3, neck_positions)